<a href="https://colab.research.google.com/github/MK316/Myapps/blob/main/random_grouping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Grouping with fixed members

In [ ]:
!pip install gradio

In [ ]:
import pandas as pd
import gradio as gr

def group_names(file, members_per_group, fixed_groups_input):
    # Read the CSV file
    df = pd.read_csv(file.name, encoding='utf-8')

    # Parse fixed groups input
    fixed_groups = [group.strip() for group in fixed_groups_input.split(';') if group.strip()]
    fixed_groups_df_list = []
    remaining_df = df.copy()

    # Process fixed groups and create a list for additional members to be added
    for group in fixed_groups:
        group_names = [name.strip() for name in group.split(',') if name.strip()]
        # Find these names in the DataFrame
        matched_rows = remaining_df[remaining_df['Names'].isin(group_names)]
        fixed_groups_df_list.append(matched_rows)
        # Remove these names from the pool of remaining names
        remaining_df = remaining_df[~remaining_df['Names'].isin(group_names)]

    # Shuffle the remaining DataFrame
    remaining_df = remaining_df.sample(frac=1).reset_index(drop=True)

    # Adjusting fixed groups to include additional members if they're under the specified group size
    for i, group_df in enumerate(fixed_groups_df_list):
        while len(group_df) < members_per_group and not remaining_df.empty:
            group_df = pd.concat([group_df, remaining_df.iloc[[0]]])
            remaining_df = remaining_df.iloc[1:].reset_index(drop=True)
        fixed_groups_df_list[i] = group_df  # Update the group with added members

    # Grouping the remaining names
    groups = fixed_groups_df_list  # Start with adjusted fixed groups
    for i in range(0, len(remaining_df), members_per_group):
        groups.append(remaining_df[i:i + members_per_group])

    # Creating a new DataFrame for grouped data
    grouped_data = []
    for i, group in enumerate(groups):
        group_name = f'Group {i+1}'
        names = ', '.join(group['Names'].tolist())
        grouped_data.append({'Group': group_name, 'Members': names})
    grouped_df = pd.DataFrame(grouped_data)

    return grouped_df

def main_interface(file, members_per_group, fixed_groups_input):
    grouped_df = group_names(file, members_per_group, fixed_groups_input)
    # Save the DataFrame to a CSV file with UTF-8 encoding
    output_filename = '/tmp/grouped_names.csv'
    grouped_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    return grouped_df, output_filename

iface = gr.Interface(
    fn=main_interface,
    inputs=[
        gr.File(label="Upload CSV File"),
        gr.Number(label="Members per Group", value=4),
        gr.Textbox(label="Fixed Groups (separated by semicolon)", placeholder="Name1, Name2; Name3, Name4", value=""),
    ],
    outputs=[gr.Dataframe(label="Grouped Names"), gr.File(label="Download Grouped Names CSV")],
    allow_flagging="never"
)

iface.launch()
